<a href="https://colab.research.google.com/github/rpscdev/tensorflow2.0/blob/main/textgeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 135.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00


In [11]:
# STEP 1: Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# STEP 2: Sample conversation-like text data
text = """
Person A: Hey, are you coming to the party tonight?
Person B: I am thinking about it, who else will be there?
Person A: Almost everyone from the office! It will be fun.
Person B: Great, I'll join. What time should I come?
Person A: Around 7 PM. See you there!
"""

# STEP 3: Tokenization
tokenizer = Tokenizer(oov_token="<OOV>")  # Add out-of-vocabulary token
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1  # +1 because of padding

# STEP 4: Create sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# STEP 5: Pad sequences
max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# STEP 6: Build model
model = Sequential([
    Embedding(total_words, 64, input_length=max_sequence_len-1),
    LSTM(150, return_sequences=True),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# STEP 7: Train model
model.fit(X, y, epochs=200, verbose=1)

# STEP 8: Text generation function
def generate_text(seed_text, next_words, model, max_sequence_len, temperature=0.8):
    output_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([output_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]

        predictions = np.log(predictions + 1e-7) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        predicted_index = np.random.choice(len(predictions), p=predictions)

        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        else:
            output_word = "<OOV>"

        output_text += ' ' + output_word

    return output_text




Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.0394 - loss: 3.6640
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0539 - loss: 3.6549 
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0643 - loss: 3.6457
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0643 - loss: 3.6303
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0747 - loss: 3.6032
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0747 - loss: 3.5716
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0539 - loss: 3.5492
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0643 - loss: 3.5293
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0747 - loss: 3.5018
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0539 - loss: 3.5126
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1286 - loss: 3.4758
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1495 - loss: 3.4161


In [12]:
import gradio as gr

def generate_response(prompt, num_words, temp):
    return generate_text(prompt, next_words=num_words, model=model, max_sequence_len=max_sequence_len, temperature=temp)

iface = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(lines=2, label="Seed Text"),
        gr.Slider(1, 50, step=1, value=10, label="Number of Words to Generate"),
        gr.Slider(0.2, 1.5, step=0.1, value=0.8, label="Temperature (Creativity)")
    ],
    outputs="text",
    title="Conversation Text Generator",
    description="Generate conversation-style text using a trained LSTM model."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a25916ed94f49450e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
